In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
bank_marketing_dataset = pd.read_csv("dataset/bank-full.csv", sep=';')

In [3]:
bank_marketing_dataset.head(n=10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [4]:
x = bank_marketing_dataset.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 15]]
y = bank_marketing_dataset['y'].values

In [5]:
x = pd.get_dummies(x, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 
                            'campaign', 'pdays', 'previous', 'poutcome'], drop_first=True)

In [6]:
x = x.values

In [7]:
# encode the output variable into 1's & 0's
label_encoder_y = LabelEncoder()
y = label_encoder_y.fit_transform(y)

In [8]:
feature_scaler = MinMaxScaler(feature_range=(0,1))
x = feature_scaler.fit_transform(x)

/anaconda/envs/deep-learning/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [10]:
# reshape Y into array of (n_x, m)
X_train = x.reshape(x.shape[1], x.shape[0])

# reshape Y into array of (n_x, m)
y_train = y.reshape(1, y.shape[0])

In [11]:
x.shape

(45211, 669)

In [16]:
def input_data(n_x, n_y):
    """
    
    """
    
    x = tf.placeholder(shape=[n_x, None], dtype=tf.float32, name='X')
    y = tf.placeholder(shape=[n_y, None], dtype=tf.float32, name='Y')
    
    return x, y

In [12]:
def initialize_params(layer_dimensions):
    """
    
    """
    
    num_of_layers = len(layer_dimensions)
    parameters = {}
    
    for l in range(1, num_of_layers):
        parameters['W' + str(l)] = tf.get_variable(shape=[layer_dimensions[l], layer_dimensions[l - 1]], 
                                                   dtype=tf.float32, name='W' + str(l))
        parameters['b' + str(l)] = tf.zeros(shape=[layer_dimensions[l], 1])
        
    return parameters

In [13]:
layer_dims = [10, 20 , 1]
derp = initialize_params(layer_dims)

In [22]:
def forward_propagation(x, parameters):
    """
    
    """
    
    A = x
    L = len(parameters) // 2
    
    for l in range(1, (L - 1)):
        Z = tf.matmul(parameters['W' + str(l)], A) + parameters['b' + str(l)]
        A = tf.nn.relu(Z)
        
    Z_output = tf.matmul(parameters['W' + str(L)], A) + parameters['b' + str(L)]
    
    return Z_output

In [15]:
def compute_cost(y_true, y_pred, parameters):
    """
    
    """
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_true))
    
    return cost

In [20]:
def train_model(x, y, parameters, num_epochs=200):
    
    n_x = x.shape[0]   #number of features for x
    n_y = x.shape[0]   #number of features for x
    m = x.shape[1]
    
    x, y = input_data(n_x, n_y)
    
    Z_output = forward_propagation(x, parameters)
    
    cost = compute_cost(Z_output, y, parameters)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        sess.run(init)
        
        loss = sess.run(cost, feed_dict={x: x, y: y})
        
    print(loss)

In [23]:
# layer_dims = [500, 250, 1]
# parameters = initialize_params(layer_dims)
train_model(X_train, y_train, derp)

ValueError: Dimensions must be equal, but are 20 and 669 for 'MatMul' (op: 'MatMul') with input shapes: [1,20], [669,?].